
#  Dicom to nifti, upscale voxel size by factor of 10 and reorientate data in atlas orientation.
This script will convert dcm files to nifti format, upscale voxel size by factor of 10 and change TR of segmented Spin-Echo SPI images to 3s.


<br>
To run this script, you need to install AFNI, FSL, and dcm2nii.
<br>
To install FSL on windows 10 see: # https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/FslInstallation/Windows
<br>
To install AFNI see: #https://afni.nimh.nih.gov/pub/dist/doc/htmldoc/background_install/install_instructs/steps_linux_ubuntu18.html
<br>
To install dcm2nii : sudo apt install mricron


-----------------------------------------------------------
Script written by Mehdi Behroozi
<br>
Biopsychology, 
<br>
Ruhr-University Bochum, Bochum, Germany
<br>
(2022.03)

-----------------------------------------------------------


In [ ]:

# Reading an excel file using Python 
import openpyxl
import os

rescale_factor=10

# Give the location of the excel file 
xlsx_file = ("/mnt/d/Data/Pigeon/Sleep/experiments_info_sleep.xlsx") 
analysis_path = "/mnt/d/Data/Pigeon/Sleep/analysis3"

# To open Workbook 
wb = openpyxl.load_workbook(xlsx_file)
sheet = wb.active 

for r in range(1,int(sheet.max_row)):
    
     if sheet.cell(r,5).value == 1:
        raw_path = sheet.cell(r,9).value
        dicom_directory = os.path.join(raw_path,str(int(sheet.cell(r,8).value)),'pdata/1/dicom')
        #print(dicom_directory)
        

        if sheet.cell(r,7).value == 'rest':
            output_path = os.path.join(analysis_path, "sub%02d"%(int(sheet.cell(r,1).value)),str(sheet.cell(r,4).value))
            
            if os.path.exists(output_path):
                print('1) The folder %s exist!'%(output_path))
                pass
            else:
                print('1) Creating folder: %s'%(output_path))
                os.makedirs(output_path)
             
            print('2) Converting dcm files to nifti file useing dcm2nii ...')
            os.system('dcm2nii -4 -g -o %s %s'%(output_path,dicom_directory))            
            files = os.listdir(output_path)
            dst = os.path.join(output_path, 'bold.nii.gz')
            out_name = 'bold'
            
        elif sheet.cell(r,7).value == 'anat':
            output_path = os.path.join(analysis_path, "sub%02d"%(int(sheet.cell(r,1).value)),str(sheet.cell(r,4).value))
            
            if os.path.exists(output_path):
                print('1) The folder %s exist!'%(output_path))
                pass
            else:
                print('1) Creating folder: %s'%(output_path))
                os.makedirs(output_path)
                
            print('2) Converting dcm files to nifti file useing dcm2nii ...')
            os.system('dcm2nii -4 -g -o %s %s'%(output_path,dicom_directory))            
            files = os.listdir(output_path)
            dst = os.path.join(output_path, 't2w.nii.gz')
            out_name = 't2w'          
            
        src = os.path.join(output_path, files[0]) 
        os.rename(src, dst)
        os.chdir(output_path)
        print('3) Upscaling voxel size by factor of %s ...'%(rescale_factor))
        os.system('3drefit -xyzscale %s %s.nii.gz'%(rescale_factor,out_name))
        
        
        print('\n\n')
        

In [ ]:
import os
import glob

from nilearn.plotting import plot_anat
%matplotlib inline
import matplotlib.pyplot as plt

data_path = '/mnt/d/Data/Pigeon/Sleep/analysis3'
anat_dirs = glob.glob('%s/*/*/t2w*'%(data_path))
for curr_anat in anat_dirs:
    #print(curr_anat)
    plot_anat(curr_anat, title=curr_anat, cut_coords = (20,20,10),
              display_mode='ortho', dim=-1, draw_cross=False, annotate=False)